# Training and deploying our Keras CNN on SageMaker

We modified our Transfer learning Keras model from the basic section to cancerDemo.py script that you must upload into this Notebook's directory. It's the same code as the previous exercise, but with a little bit of extra stuff to allow hyperparameters to be passed in as arguments from SageMaker.

In [ ]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

## Download dataset to disk
Downloading data from github to disk


In [ ]:
import os
import numpy as np
import requests
import zipfile

file_url = "https://github.com/satoru2001/Cancer_Dataset/raw/master/dataset.zip"
    
r = requests.get(file_url, stream = True)

#Downloading data
with open("dataset.zip", "wb") as file:  
  for block in r.iter_content(chunk_size = 1024): 
    if block:
      file.write(block)

#Extracting zip file
import zipfile
loc_ref = "dataset.zip"
zip_ref = zipfile.ZipFile(loc_ref)
zip_ref.extractall()
zip_ref.close()

## Uploading data to S3
As EC2 instance need data to be fed in through S3

In [ ]:
cwd = os.getcwd()
training_folder = os.path.join(cwd,"tmp/Train")
prefix = 'cancer'

training_input_path   = sess.upload_data(training_folder, key_prefix=prefix)

In [ ]:
print(training_input_path)

## Train our instance for single epoch to test every thing is correct before proceeding further

We'll test out running a single epoch, just to make sure the script works before we start spending money on other instances. Here as it is a demo you can skip train on a GPU cell as it costs money.

As we are training for 1 epoch the accuracy will be less

If you want to train for more epochs change the epoch parameter in the code

In [ ]:
!pygmentize cancerDemo.py #To display our cancerDemo.py

In [ ]:
from sagemaker.tensorflow import TensorFlow

#Train a custom TensorFlow model in local instance.
tf_estimator = TensorFlow(entry_point='cancerDemo.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='2.0', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )

In [ ]:
tf_estimator.fit({'training': training_input_path})

## Train on a GPU instance with 10 epochs

Now that we're sure it works, you can use gpu's to train<br>
**Note** It costs money so I commented if any one want to try you can uncomment and run it

In [ ]:
#train a custom TensorFlow model in P3 instance
#tf_estimator = TensorFlow(entry_point='mnist-train-cnn.py', 
#                           role=role,
#                           train_instance_count=1, 
#                           train_instance_type='ml.p3.2xlarge',
#                           framework_version='2.00', 
#                           py_version='py3',
#                           script_mode=True,
#                           hyperparameters={'epochs': 10}
#                          )

In [ ]:
# tf_estimator.fit({'training': training_input_path})

## Deploy the model

### For this we have plenty of options
1. Choose GPU instance as above but it costs much
1. choose High end CPU instance + Elastic Inference accelerator which costs significantly less with high performence
1. Use our local instance as this is not a complex algo

we will go with 3 as it doesn't cost any money.

If you want to try second one use
```
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                         instance_type='ml.c5.large',        
                         accelerator_type='ml.eia1.medium',  
                         endpoint_name=tf_endpoint_name) 
```
instead of below code for tf_predictor

In [ ]:
import time

tf_endpoint_name = 'keras-tf-mnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

#Creating an endpoint to work deploy model
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                         instance_type='local',  
                         endpoint_name=tf_endpoint_name)     

## Make predictions with the deployed model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img,img_to_array
num_samples = 5
train_dir = os.path.join(cwd,"tmp/Train")
IDC = os.path.join(train_dir,'IDC')
NONIDC = os.path.join(train_dir,'NONIDC')
prediction_array = []
for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)#Plots
    img = load_img(IDC + "/" +os.listdir(IDC)[i], target_size=(50, 50))#fetch image from directory
    prediction_array.append(img_to_array(img))#Store image array need to be sen for predictions
    plt.imshow(img)#Display image
    
prediction_array = np.array(prediction_array)
prediction_array.shape
prediction = tf_predictor.predict(prediction_array)['predictions']#Sending array for predictions
prediction = np.array(prediction)
for i in prediction:
    print("NONIDC") if i>0.5 else print("IDC")
print("Here we passed all IDC i.e Positive as we train our model only once our model cannot predict every thing correctly")

## Clean up the deployment endpoint
If you are using other than local instance for deployment

you can delete endpoint from console also in Sagemaker

Under Inference, choose Endpoints.

Choose the endpoint that you created in the example, then choose Actions | Delete.

Choose Delete.

In [ ]:
sess.delete_endpoint(endpoint_name=tf_endpoint_name)

## Remember to shut down your notebook instance too!